## Closed-loop batch, constrained BO in BoTorch with qEI and qNEI

In this tutorial, we illustrate how to implement a simple Bayesian Optimization (BO) closed loop in BoTorch.

In general, we recommend for a relatively simple setup (like this one) to use Ax, since this will simplify your setup (including the amount of code you need to write) considerably. See the [Using BoTorch with Ax](./custom_botorch_model_in_ax) tutorial.

However, you may want to do things that are not easily supported in Ax at this time (like running high-dimensional BO using a VAE+GP model that you jointly train on high-dimensional input data). If you find yourself in such a situation, you will need to write your own optimization loop, as we do in this tutorial.


We use the batch Expected Improvement (qEI) and batch Noisy Expected Improvement (qNEI) acquisition functions to optimize a constrained version of the synthetic Hartmann6 test function. The standard problem is

$$f(x) = -\sum_{i=1}^4 \alpha_i \exp \left( -\sum_{j=1}^6 A_{ij} (x_j - P_{ij})^2  \right)$$

over $x \in [0,1]^6$ (parameter values can be found in `botorch/test_functions/hartmann6.py`).

In real BO applications, the design $x$ can influence multiple metrics in unknown ways, and the decision-maker often wants to optimize one metric without sacrificing another. To illustrate this, we add a synthetic constraint fo the form $\|x\|_1 - 3 \le 0$. Both the objective and the constraint are observed with noise. 

Since botorch assumes a maximization problem, we will attempt to maximize $-f(x)$ to achieve $\max_{x} -f(x) = 3.32237$.

In [1]:
from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qExpectedImprovement, qNoisyExpectedImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning
import time

import warnings
# warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
# warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings("ignore")



In [2]:
import torch
torch.manual_seed(0)
device = torch.device("cpu")#("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

### Problem setup

First, we define the constraint used in the example in `outcome_constraint`. The second function `weighted_obj` is a "feasibility-weighted objective," which returns zero when not feasible. 

In [3]:
from botorch.test_functions.synthetic import Hartmann

neg_hartmann6 = Hartmann(negate=True)

In [4]:
from contextlib import contextmanager
# Helper functions
@contextmanager
def timer(label):
    import time
    start = time.time()
    yield
    print(f"[Process {label}] elasped in {time.time()-start}")
    

In [5]:
def outcome_constraint(X):
    """L1 constraint; feasible if less than or equal to zero."""
    return X.sum(dim=-1) - 3

def weighted_obj(X):
    """Feasibility weighted objective; zero if not feasible."""
    return neg_hartmann6(X) * (outcome_constraint(X) <= 0).type_as(X)

#### Model initialization

We use a `MultiOutputGP` to model the objective (output 0) and the constraint (output 1). We assume known homoskedastic observation noise on both the objective and constraint with standard error $\sigma = 0.5$. 

Each component is a `FixedNoiseGP`. The models are initialized with 10 points drawn randomly from $[0,1]^6$.

In [6]:
from botorch.models import FixedNoiseGP, ModelListGP, HeteroskedasticSingleTaskGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood

NOISE_SE = 0.5
NOISE_OF_NOISE = 0.1
NOISE_FURTHER = 0.2


def generate_initial_data(n=10):
    train_yvar = []
    # generate training data
    train_x = torch.rand(n, 6, device=device, dtype=dtype)
    print(train_x.shape)
    exact_obj = neg_hartmann6(train_x).unsqueeze(-1)  # add output dimension
    print(exact_obj.shape)
    exact_con = outcome_constraint(train_x).unsqueeze(-1)  # add output dimension
    observed_noise = NOISE_SE * torch.randn_like(exact_obj) + NOISE_OF_NOISE * torch.randn_like(exact_obj)
    train_obj = exact_obj + observed_noise + NOISE_FURTHER * torch.randn_like(exact_obj)
    train_con = exact_con + NOISE_SE * torch.randn_like(exact_con)
    best_observed_value = weighted_obj(train_x).max().item()
    train_yvar.append(observed_noise)
    return train_x, train_obj, train_con, best_observed_value, train_yvar
    
    
def initialize_model(train_x, train_obj, train_con, train_yvar, state_dict=None):
    train_yvar_con = torch.tensor(NOISE_SE**2, device=device, dtype=dtype)
    train_yvar_obj = torch.cat(train_yvar) ** 2
    # define models for objective and constraint
    model_obj = HeteroskedasticSingleTaskGP(train_x, train_obj, train_yvar_obj).to(train_x)
    model_con = HeteroskedasticSingleTaskGP(train_x, train_con, train_yvar_con.expand_as(train_con)).to(train_x)
    # combine into a multi-output GP model
    model = ModelListGP(model_obj, model_con)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    # load state dict if it is passed
    if state_dict is not None:
        model.load_state_dict(state_dict)
    return mll, model

#### Define a construct to extract the objective and constraint from the GP
The methods below take the outputs of the GP and return the objective and the constraint. In general, these can be any `Callable`, but here we simply need to index the correct output.

In [7]:
from botorch.acquisition.objective import ConstrainedMCObjective

def obj_callable(Z):
    return Z[..., 0]

def constraint_callable(Z):
    return Z[..., 1]

# define a feasibility-weighted objective for optimization
constrained_obj = ConstrainedMCObjective(
    objective=obj_callable,
    constraints=[constraint_callable],
)

In [8]:
import numpy as np
np.random.uniform(0, 1)

0.29379725788967226

#### Define a helper function that performs the essential BO step
The helper function below takes an acquisition function as an argument, optimizes it, and returns the batch $\{x_1, x_2, \ldots x_q\}$ along with the observed function values. For this example, we'll use a small batch of $q=3$. The function `optimize_acqf` optimizes the $q$ points jointly. A simple initialization heuristic is used to select the 10 restart initial locations from a set of 50 random points. 

In [68]:
from botorch.optim import optimize_acqf
from gpytorch.utils.errors import NanError


BATCH_SIZE = 3
bounds = torch.tensor([[0.0] * 6, [1.0] * 6], device=device, dtype=dtype)

# def get_observation()

def optimize_acqf_and_get_observation(acq_func):
    """Optimizes the acquisition function, and returns a new candidate and a noisy observation."""
    
#     with timer("Generate candidate"):
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=BATCH_SIZE,
        num_restarts=10,
        raw_samples=500,  # Sample on GP using Sobel sequence
        options={
            "batch_limit": 5,
            "max_iter": 200,
            "seed": 0
        }
    )      
            
    train_yvar = []
    # observe new values <-- new values -> obtained in batches?
    new_x = candidates.detach()
#     print(new_x.shape)
    exact_obj = neg_hartmann6(new_x).unsqueeze(-1)  # add output dimension
    exact_con = outcome_constraint(new_x).unsqueeze(-1)  # add output dimension
    observed_noise = NOISE_SE * torch.randn_like(exact_obj) + NOISE_OF_NOISE * torch.randn_like(exact_obj)
    new_obj = exact_obj + observed_noise + NOISE_FURTHER * torch.randn_like(exact_obj)
    new_con = exact_con + NOISE_SE * torch.randn_like(exact_con)
    train_yvar.append(observed_noise)
    return new_x, new_obj, new_con, train_yvar


def update_random_observations(best_random):
    """Simulates a random policy by taking a the current list of best values observed randomly,
    drawing a new random point, observing its value, and updating the list.
    """
    rand_x = torch.rand(BATCH_SIZE, 6)
    next_random_best = weighted_obj(rand_x).max().item()
    best_random.append(max(best_random[-1], next_random_best))       
    return best_random

In [69]:
bounds

tensor([[0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.]], dtype=torch.float64)

### Perform Bayesian Optimization loop with qNEI
The Bayesian optimization "loop" for a batch size of $q$ simply iterates the following steps:
1. given a surrogate model, choose a batch of points $\{x_1, x_2, \ldots x_q\}$
2. observe $f(x)$ for each $x$ in the batch 
3. update the surrogate model. 


Just for illustration purposes, we run three trials each of which do `N_BATCH=20` rounds of optimization. The acquisition function is approximated using `MC_SAMPLES=500` samples.

*Note*: Running this may take a little while.

In [10]:
from copy import copy

N_TRIALS = 1
N_BATCH = 1
MC_SAMPLES = 500

verbose = True

best_observed_all_ei, best_observed_all_nei, best_random_all = [], [], []

# average over multiple trials
for trial in range(1, N_TRIALS + 1):
    
    print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
    best_observed_ei, best_observed_nei, best_random = [], [], []
    
    # call helper functions to generate initial training data and initialize model
    train_x_ei, train_obj_ei, train_con_ei, best_observed_value_ei, train_yvar_ei = generate_initial_data(n=500)
    mll_ei, model_ei = initialize_model(train_x_ei, train_obj_ei, train_con_ei, train_yvar_ei)
    
    train_x_nei, train_obj_nei, train_con_nei, train_yvar_nei = train_x_ei, train_obj_ei, train_con_ei, copy(train_yvar_ei)
    best_observed_value_nei = best_observed_value_ei
    mll_nei, model_nei = initialize_model(train_x_nei, train_obj_nei, train_con_nei, train_yvar_nei)
    
    best_observed_ei.append(best_observed_value_ei)
    best_observed_nei.append(best_observed_value_nei)
    best_random.append(best_observed_value_ei)
    
    # run N_BATCH rounds of BayesOpt after the initial random batch
    for iteration in range(1, N_BATCH + 1):    
        
        t0 = time.time()
        
        # fit the models
#         fit_gpytorch_model(mll_ei)
        fit_gpytorch_model(mll_nei)
        
        # define the qEI and qNEI acquisition modules using a QMC sampler
        qmc_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES, seed=0)
        
        # for best_f, we use the best observed noisy values as an approximation
#         qEI = qExpectedImprovement(
#             model=model_ei, 
#             best_f=(train_obj_ei * (train_con_ei <= 0).to(train_obj_ei)).max(),
#             sampler=qmc_sampler, 
#             objective=constrained_obj,
#         )
        
        qNEI = qNoisyExpectedImprovement(
            model=model_nei, 
            X_baseline=train_x_nei,
            sampler=qmc_sampler, 
            objective=constrained_obj,
        )
        
        # optimize and get new observation
#         new_x_ei, new_obj_ei, new_con_ei, new_yvar_ei = optimize_acqf_and_get_observation(qEI)
        new_x_nei, new_obj_nei, new_con_nei, new_yvar_nei = optimize_acqf_and_get_observation(qNEI)
                
        # update training points
#         train_x_ei = torch.cat([train_x_ei, new_x_ei])
#         train_obj_ei = torch.cat([train_obj_ei, new_obj_ei])
#         train_con_ei = torch.cat([train_con_ei, new_con_ei])
#         train_yvar_ei.extend(new_yvar_ei)

        train_x_nei = torch.cat([train_x_nei, new_x_nei])
        train_obj_nei = torch.cat([train_obj_nei, new_obj_nei])
        train_con_nei = torch.cat([train_con_nei, new_con_nei])
        train_yvar_nei.extend(new_yvar_nei)

        # update progress
#         best_random = update_random_observations(best_random)
#         best_value_ei = weighted_obj(train_x_ei).max().item()
        best_value_nei = weighted_obj(train_x_nei).max().item()
#         best_observed_ei.append(best_value_ei)
        best_observed_nei.append(best_value_nei)

        # reinitialize the models so they are ready for fitting on next iteration
        # use the current state dict to speed up fitting
#         mll_ei, model_ei = initialize_model(
#             train_x_ei, 
#             train_obj_ei, 
#             train_con_ei,
#             train_yvar_ei,
#             model_ei.state_dict(),
#         )
        mll_nei, model_nei = initialize_model(
            train_x_nei, 
            train_obj_nei, 
            train_con_nei,
            train_yvar_nei,
            model_nei.state_dict(),
        )
        
        t1 = time.time()
        
        if verbose:
            print(
                f"\nBatch {iteration:>2}: best_value (random, qEI, qNEI) = "
                #f"({max(best_random):>4.2f}, {best_value_ei:>4.2f}, 
                f"{best_value_nei:>4.2f}), "
                f"time = {t1-t0:>4.2f}.", end=""
            )
        else:
            print(".", end="")
   
    best_observed_all_ei.append(best_observed_ei)
    best_observed_all_nei.append(best_observed_nei)
    best_random_all.append(best_random)


Trial  1 of 1 torch.Size([500, 6])
torch.Size([500, 1])

Batch  1: best_value (random, qEI, qNEI) = 3.16), time = 131.46.

In [11]:
mll_ei, model_ei = initialize_model(
    train_x_ei, 
    train_obj_ei, 
    train_con_ei,
    train_yvar_ei,
    model_ei.state_dict(),
)

# fit the models
# fit_gpytorch_model(mll_ei)

# define the qEI and qNEI acquisition modules using a QMC sampler
qmc_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES, seed=0)

# for best_f, we use the best observed noisy values as an approximation
qEI = qExpectedImprovement(
    model=model_ei, 
    best_f=(train_obj_ei * (train_con_ei <= 0).to(train_obj_ei)).max(),
    sampler=qmc_sampler, 
    objective=constrained_obj,
)

# optimize and get new observation
optimize_acqf_and_get_observation(qEI)

(tensor([[0.9081, 0.4792, 0.4013, 0.4520, 0.8302, 0.6463],
         [0.6200, 0.9347, 0.2195, 0.7131, 0.6599, 0.3554],
         [0.1725, 0.1999, 0.4361, 0.1504, 0.1866, 0.7859]], dtype=torch.float64),
 tensor([[0.2415],
         [0.3799],
         [1.6446]], dtype=torch.float64),
 tensor([[ 0.9606],
         [ 0.6880],
         [-1.3711]], dtype=torch.float64),
 [tensor([[ 0.2367],
          [-0.2031],
          [-0.2133]], dtype=torch.float64)])

In [91]:
# Test for the neioptimizer
from gpytorch.mlls import ExactMarginalLogLikelihood


def initialize_model(candidate, training_result, state_dict=None):
    global train_x, train_obj, train_var
    # Group candidates, objectives and variances and put into a torch Tensor
    train_x, train_obj, train_var = [], [], []
    for o in observations:
        train_x.append(list(o["candidate"].values()))
        train_obj.append(o["result"][0])
        train_var.append(o["result"][1])
    train_x.append(list(candidate.values()))
    train_obj.append(training_result[0])
    train_var.append(training_result[1])
    train_x = torch.tensor(train_x, device=device, dtype=dtype)
    train_obj = torch.tensor(train_obj, device=device, dtype=dtype).unsqueeze(-1)
    train_var = torch.tensor(train_var, device=device, dtype=dtype).unsqueeze(-1)

    print(f"X: {train_x.shape}, y: {train_obj.shape}, var: {train_var.shape}")
    # define models for objective and constraint
    model = HeteroskedasticSingleTaskGP(train_x, train_obj, train_var**2).to(train_x)
    # combine into a multi-output GP model
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    # load state dict if it is passed
    if state_dict is not None:
        model.load_state_dict(state_dict)
    return mll, model

observations = [
        {"candidate": {"x1":0.36589372, "x2":0.16217732, "x3":0.3572523, "x4":0.08772629, "x5":0.47287726, "x6":0.9107906},
         "result": (1.0175396, -0.10288623)},
        {"candidate": {"x1":0.2034043, "x2":0.5584821, "x3":0.78182673, "x4":0.07152402, "x5":0.88407385, "x6":0.46330553},
        'result': (-0.14934252, -0.23847157)}
]
trainer_info = {'result': (0.5329527, 0.15520538)}
candidate = {"x1":0.5, "x2":0.5, "x3":0.5, "x4":0.5, "x5":0.5, "x6":0.6}
bounds = {"x1": (0, 1), "x2": (0, 1), "x3": (0, 1), "x4": (0, 1), "x5": (0, 1), "x6": (0, 1)}


lower_bounds = [bound[0] for bound in bounds.values()]
upper_bounds = [bound[1] for bound in bounds.values()]
torch_bounds = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=dtype)
mll, model = initialize_model(candidate, trainer_info["result"])
fit_gpytorch_model(mll)
qmc_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES, seed=0)
qNEI = qNoisyExpectedImprovement(
    model=model, 
    X_baseline=train_x,
    sampler=qmc_sampler, 
)
torch_candidate, _ = optimize_acqf(
    acq_function=qNEI,
    bounds=torch_bounds,
    q=1,
    num_restarts=10,
    raw_samples=500,  # Sample on GP using Sobel sequence
    options={
        "batch_limit": 5,
        "max_iter": 200,
        "seed": 0
    }
)    

candidate = {}
for i, key in enumerate(bounds.keys()):
    candidate[key] = torch_candidate.cpu().numpy()[0][i]
candidate

X: torch.Size([3, 6]), y: torch.Size([3, 1]), var: torch.Size([3, 1])


{'x1': 0.47448268759621093,
 'x2': 0.06570019846549614,
 'x3': 0.1983489942378863,
 'x4': 0.19961448365460774,
 'x5': 0.2908090385295882,
 'x6': 1.0}

In [103]:
from typing import *

observations = [
        {"candidate": {"x1":0.36589372, "x2":0.16217732, "x3":0.3572523, "x4":0.08772629, "x5":0.47287726, "x6":0.9107906},
         "result": (1.0175396, -0.10288623)},
        {"candidate": {"x1":0.2034043, "x2":0.5584821, "x3":0.78182673, "x4":0.07152402, "x5":0.88407385, "x6":0.46330553},
        'result': (-0.14934252, -0.23847157)}
]
trainer_info = {'result': (0.5329527, 0.15520538)}
candidate = {"x1":0.5, "x2":0.5, "x3":0.5, "x4":0.5, "x5":0.5, "x6":0.6}
bounds = {"x1": (0, 1), "x2": (0, 1), "x3": (0, 1), "x4": (0, 1), "x5": (0, 1), "x6": (0, 1)}

def initialize_model(candidate, training_result, state_dict=None):
    r""" TODO

    :param candidate: 
    """

    # Group candidates, objectives and variances from observations 
    # and put into a torch Tensor
    train_x, train_obj, train_var = [], [], []
    for o in observations:
        train_x.append(list(o["candidate"].values()))
        train_obj.append(o["result"][0])
        train_var.append(o["result"][1])
    train_x.append(list(candidate.values()))
    train_obj.append(training_result[0])
    train_var.append(training_result[1])
#     print(train_x)
    train_x = torch.tensor(train_x, device=device, dtype=dtype)
    train_obj = torch.tensor(train_obj, device=device, dtype=dtype).unsqueeze(-1)
    train_var = torch.tensor(train_var, device=device, dtype=dtype).unsqueeze(-1)

    # define models for objective and constraint
    model = HeteroskedasticSingleTaskGP(train_x, train_obj, train_var**2).to(train_x)
    # combine into a multi-output GP model
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    # load state dict if it is passed
    if state_dict is not None:
        model.load_state_dict(state_dict)
    return mll, model

# TODO: Change model initialization upon NanError thrown by cholesky decomposition
def generate_candidate(candidate: Dict[str, Any], trainer_info: Union[Dict, None]) \
        -> Dict[str, Any]:
    if trainer_info is None and len(observations) == 0:
        # Generate a random candidate to startup the optimizing process
        return self._generate_random_candidate()
    elif trainer_info is None and len(observations) > 0:
        # TODO: Need to handle this case
        pass
    else:
        mll, model = initialize_model(candidate, trainer_info["result"])
        fit_gpytorch_model(mll)
        qmc_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES, seed=0)
        qNEI = qNoisyExpectedImprovement(
            model=model, 
            X_baseline=torch.tensor([list(o["candidate"].values()) for o in observations], device=device, dtype=dtype),
            sampler=qmc_sampler, 
        )

        # Torch based bounds
        lower_bounds = [bound[0] for bound in bounds.values()]
        upper_bounds = [bound[1] for bound in bounds.values()]
        bounds_torch = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=dtype)
        torch_candidate, _ = optimize_acqf(
            acq_function=qNEI,
            bounds=bounds_torch,
            q=1,              # Generate only one candidate at a time
            num_restarts=10,  # ???
            raw_samples=500,  # Sample on GP using Sobel sequence
            options={
                "batch_limit": 5,
                "max_iter": 200,
                "seed": 0
            }
        )  

        candidate = {}
        for i, key in enumerate(bounds.keys()):
            candidate[key] = torch_candidate.cpu().numpy()[0][i]

        return candidate
    
generate_candidate(candidate, trainer_info)

{'x1': 0.4745468242989644,
 'x2': 0.06562127492160173,
 'x3': 0.19841943193999256,
 'x4': 0.1996072423922306,
 'x5': 0.29071384478280604,
 'x6': 1.0}

In [100]:
[list(o["candidate"])]

[{'candidate': {'x1': 0.36589372,
   'x2': 0.16217732,
   'x3': 0.3572523,
   'x4': 0.08772629,
   'x5': 0.47287726,
   'x6': 0.9107906},
  'result': (1.0175396, -0.10288623)},
 {'candidate': {'x1': 0.2034043,
   'x2': 0.5584821,
   'x3': 0.78182673,
   'x4': 0.07152402,
   'x5': 0.88407385,
   'x6': 0.46330553},
  'result': (-0.14934252, -0.23847157)}]

In [42]:
from typing import Dict, Any

candidate = {"x1":0.5, "x2":0.5, "x3":0.5, "x4":0.5, "x5":0.5, "x6":0.6}

def get_observation(candidate: Dict[str, Any]) \
            -> Dict[str, Any]:
        r""" Get observation by plugging the candidate into objective function.
        This method is made abstract to easier modify the objective function
        to run on different platforms.

        :param candidate:
        :return:
        """
        objective_function = Hartmann(negate=True)
        train_x = torch.rand(6, device=device)
        exact_obj = objective_function(train_x).unsqueeze(-1)  # add output dimension
        # Add noise to the objective function
        observed_noise = NOISE_SE * torch.randn_like(exact_obj) + NOISE_OF_NOISE * torch.randn_like(exact_obj)
        train_obj = exact_obj + observed_noise + NOISE_FURTHER * torch.randn_like(exact_obj)
        print(train_x.cpu().numpy())
        return {"results": (train_obj.cpu().numpy()[0], observed_noise.cpu().numpy()[0])}
    
get_observation(candidate)

[0.2034043  0.5584821  0.78182673 0.07152402 0.88407385 0.46330553]


{'results': (-0.14934252, -0.23847157)}

#### Plot the results
The plot below shows the best objective value observed at each step of the optimization for each of the algorithms. The confidence intervals represent the variance at that step in the optimization across the trial runs.

In [12]:
import numpy as np

GLOBAL_MAXIMUM = neg_hartmann6.optimal_value

from matplotlib import pyplot as plt
%matplotlib widget

def ci(y):
    return 1.96 * y.std(axis=0) / np.sqrt(N_TRIALS)

iters = np.arange(N_BATCH + 1) * BATCH_SIZE
y_ei = np.asarray(best_observed_all_ei)
y_nei = np.asarray(best_observed_all_nei)
y_rnd = np.asarray(best_random_all)

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.errorbar(iters, y_rnd.mean(axis=0), yerr=ci(y_rnd), label="random", linewidth=1.5)
ax.errorbar(iters, y_ei.mean(axis=0), yerr=ci(y_ei), label="qEI", linewidth=1.5)
ax.errorbar(iters, y_nei.mean(axis=0), yerr=ci(y_nei), label="qNEI", linewidth=1.5)
plt.plot([0, N_BATCH * BATCH_SIZE], [GLOBAL_MAXIMUM] * 2, 'k', label="true best bjective", linewidth=2)
ax.set_ylim(bottom=0.5)
ax.set(xlabel='number of observations (beyond initial points)', ylabel='best objective value')
ax.legend(loc="lower right")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AssertionError: 